In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv("data_cleaned.csv")

In [3]:
data = data.drop(data.columns[0], axis=1)

In [4]:
data = data.dropna(subset=['b_per'])

In [33]:
data.head()

,sentence,b_per,i_per,sentence_normalized,tokenizer,key_words,score
0,"Aaargh), but the fact that the raw material is...",Aaargh,Grisham,aaargh but the fact that the raw material is a...,"[aaargh, but, the, fact, that, the, raw, mater...",aaargh fact raw material john grisham tale exc...,0.023077
1,I cannot say that Aag is the worst Bollywood f...,Aag,NaN,i cannot say that aag is the worst bollywood f...,"[i, cannot, say, that, aag, is, the, worst, bo...",cannot aag worst bollywood film havent seen bo...,0.027778
2,And if you valued music more than story the li...,Aag,Naam,and if you valued music more than story the li...,"[and, if, you, valued, music, more, than, stor...",valued music story list simply grow beautiful ...,0.416667
3,What Aag does is take these characters and mes...,Aag,NaN,what aag does is take these characters and mes...,"[what, aag, does, is, take, these, characters,...",aag characters mess badly ull need contest pic...,-0.625000
4,Enough of stupid characters in your movie like...,Aag,NaN,enough of stupid characters in your movie like...,"[enough, of, stupid, characters, in, your, mov...",stupid characters movie like shiva aag,-0.800000


In [6]:
data.to_csv("data_cleaned.csv")

## clean_sentence

In [12]:
import re

def clean_text(text):

    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    text = text.lower()  # Convert text to lowercase
    return text

In [13]:
data['sentence_normalized'] = data['sentence'].apply(lambda x: clean_text(x))

## tokenizer

In [14]:
tokenizer = []  
for sentence in data["sentence_normalized"].astype(str):  # Ensure all reviews are strings
    # Tokenize the review: remove non-alphanumeric characters and split into words
    word_tokens = sentence.split()
    tokenizer.append(word_tokens)

In [15]:
data["tokenizer"]=tokenizer

In [18]:
from spacy.lang.en.stop_words import STOP_WORDS

In [22]:
deselect_stop_words = ['n\'', 'love', 'hate', 'like', 'dislike', 'not', '’t', 'never', 'nobody', 'n’t','cannot','nothing','no']
stop_words = set(STOP_WORDS)
for w in deselect_stop_words:
    if w in stop_words:
        stop_words.remove(w)
    else:
         continue

# Now, stop_words contains the default stopwords excluding the ones we deselected
print(stop_words)

{'four', 'same', 'well', 'anyway', 'him', 'afterwards', 'will', 'out', 'mine', 'either', 'nine', 'therein', 'made', 'who', 'you', 'have', 'i', 'below', 'these', 'give', 'due', 'everything', 'two', 'from', 'them', 'least', 'thereby', 'five', 'anyhow', 'each', 'under', 'again', 'been', 'for', 'becoming', '‘m', 'there', 'side', 'whether', 'say', 'various', 'somewhere', 'regarding', 'noone', 'it', 'go', 'he', 'whole', 'their', '’ll', 'himself', 'hereafter', 'quite', 'hereupon', 'though', 'with', 'ours', '‘re', 'seeming', 'eleven', 'less', 'many', 'whom', 'of', 'beside', 'here', 'without', 'seem', 'everyone', 'but', 'is', 'however', 'eight', 'therefore', 'since', 'among', 'none', 'whatever', 'before', 'during', 'me', 'get', 'more', 'herein', '’ve', 'whoever', 'others', 'are', 'anywhere', 'as', 'six', 'yet', 'toward', 'an', 'over', 'thence', 'even', 'every', 'while', 'us', 'top', 'name', 'done', 'such', 'between', 'using', 'elsewhere', 'now', 'call', 'along', 'something', 'which', '‘s', "'ll

In [23]:
key_words = []
for comment in data["tokenizer"]:
    filteredComment = [w for w in comment if not (w in stop_words or len(w) <= 1)]  # Remove stop words and short words (<=2)
    key_words.append(' '.join(filteredComment))

In [28]:
data["key_words"]=key_words

In [29]:
from textblob import TextBlob

In [31]:
score = []
for i in data["key_words"]:
    score.append(TextBlob(i).sentiment.polarity)

In [32]:
data["score"]=score

In [35]:
data.to_csv("data_score.csv")

In [37]:

def get_central_sentence(group):
    # Find the median score for the group
    median_score = group['score'].median()
    # Calculate the absolute difference between the score and the median score
    group['abs_diff'] = (group['score'] - median_score).abs()
    # Find the row with the smallest absolute difference
    central_sentence = group.loc[group['abs_diff'].idxmin()]
    return central_sentence[['b_per', 'sentence', 'score']]

# Apply the function to each group
result = data.groupby('b_per').apply(get_central_sentence).reset_index(drop=True)

# Display the result
print(result)


         b_per                                           sentence     score
0       Aaargh  Aaargh), but the fact that the raw material is...  0.023077
1          Aag  I cannot say that Aag is the worst Bollywood f...  0.027778
2         Aage  and Robert Lloyd and Aage Haugland both play a... -0.050000
3       Aakash  I watched this movie on march 21 this year.Mus...  0.156731
4      Aakrosh  After 'Aakrosh' , this was the second film for...  0.000000
...        ...                                                ...       ...
16346     yeon  I know the actress Do-yeon Jeon got the Cannes...  1.000000
16347  yiannis  Perhaps, one of the most important and enjoyab...  0.313393
16348      you  If you are interested in making a reality film...  0.225000
16349     yway  ANyway, eventually Batman discovers that he's ...  0.000000
16350       zo  But when he himself is hustled by a scraggy, w...  0.084375

[16351 rows x 3 columns]


In [38]:
result.head()

,b_per,sentence,score
0,Aaargh,"Aaargh), but the fact that the raw material is...",0.023077
1,Aag,I cannot say that Aag is the worst Bollywood f...,0.027778
2,Aage,and Robert Lloyd and Aage Haugland both play a...,-0.050000
3,Aakash,I watched this movie on march 21 this year.Mus...,0.156731
4,Aakrosh,"After 'Aakrosh' , this was the second film for...",0.000000


In [39]:
result.to_csv("Final.csv")